In [21]:
! pip install requests pandas
import requests
import pandas as pd
from datetime import date


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# import requests

# def fecth_post_from_reddit(subreddit: str, categories: list = ['hot'], maximum_post: int = 100) -> list:
#     '''
#     subreddit: name of subreddit. For example: Python
#     categories: List of categories to fetch posts from. For example: ['hot', 'new', 'top', 'rising', 'controversial']
#     maximum_post: maximum number of posts to fetch. -1 for all posts. minimum is 100
#     '''
#     headers = {"User-Agent": "post-crawler"}
#     params = {"limit": 100}
#     posts = []

#     for category in categories:
#         url = f"https://www.reddit.com/r/{subreddit}/{category}.json"
        
#         while True:
#             response = requests.get(url, headers=headers, params=params)

#             if response.status_code != 200:
#                 print(f"❌ Failed to fetch from {url}")
#                 print(f"Status code: {response.status_code}")
#                 print(f"Response text: {response.text[:200]}")  # Print part of the response for debug
#                 break

#             try:
#                 data = response.json()
#             except requests.exceptions.JSONDecodeError:
#                 print("❌ JSON decode error")
#                 print(f"Response text: {response.text[:200]}")
#                 break

#             posts.extend(data.get('data', {}).get('children', []))

#             after = data.get('data', {}).get('after')
#             if not after or (maximum_post != -1 and len(posts) >= maximum_post):
#                 break

#             params['after'] = after  # Set for next pagination

#         # Reset 'after' for the next category
#         params.pop('after', None)

#     post_datas = [post['data'] for post in posts]
#     return post_datas[:maximum_post] if maximum_post != -1 else post_datas


Scrape posts and Process DataFrame

In [ ]:
# subreddits = ['creepypasta', 'nosleep', 'shortscarystories']

# for subreddit in subreddits:
#     posts = fecth_post_from_reddit(subreddit, ['hot', 'new', 'top', 'rising', 'controversial'], maximum_post=-1)
#     posts_df = pd.DataFrame(posts)
#     posts_df.drop_duplicates(subset=['title'], inplace=True)  # Drop duplicates

#     print(f"{subreddit}: {posts_df.shape}")

#     # Print column names and the first row
#     print("Columns:", posts_df.columns.tolist())
#     if not posts_df.empty:
#         print("First row:", posts_df.iloc[0].tolist())

#     today = str(date.today()).replace('-', '_')
#     posts_df.to_csv(f'./{subreddit}/{subreddit}_{today}.csv', index=False)

❌ Failed to fetch from https://www.reddit.com/r/creepypasta/hot.json
Status code: 403
Response text: <body class=theme-beta><div><style>.theme-light,:root{--rem360:22.5rem;--rem320:20rem;--rem192:12rem;--rem144:9rem;--rem128:8rem;--rem96:6rem;--rem90:5.625rem;--rem88:5.5rem;--rem64:4rem;--rem56:3.5re
❌ Failed to fetch from https://www.reddit.com/r/creepypasta/new.json
Status code: 403
Response text: <body class=theme-beta><div><style>.theme-light,:root{--rem360:22.5rem;--rem320:20rem;--rem192:12rem;--rem144:9rem;--rem128:8rem;--rem96:6rem;--rem90:5.625rem;--rem88:5.5rem;--rem64:4rem;--rem56:3.5re
❌ Failed to fetch from https://www.reddit.com/r/creepypasta/top.json
Status code: 403
Response text: <body class=theme-beta><div><style>.theme-light,:root{--rem360:22.5rem;--rem320:20rem;--rem192:12rem;--rem144:9rem;--rem128:8rem;--rem96:6rem;--rem90:5.625rem;--rem88:5.5rem;--rem64:4rem;--rem56:3.5re
❌ Failed to fetch from https://www.reddit.com/r/creepypasta/rising.json
Status code: 403
Respo

KeyboardInterrupt: 

In [3]:
import pandas as pd
import praw
import os
from dotenv import load_dotenv
load_dotenv()

# Initialize Reddit instance
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
)
subreddits = ['creepypasta', 'nosleep', 'shortscarystories']

In [30]:
def get_url(subreddit_name: str, fetch_hot: bool=True, fetch_new: bool =True, 
            fetch_top: bool = True, fetch_rising: bool = True, fetch_controversial: bool = True,
            maximum_post: int = 100) -> list:
    '''
    subreddit: name of subreddit. For example: Python
    categories: List of categories to fetch posts from. For example: ['hot', 'new', 'top', 'rising', 'controversial']
    maximum_post: maximum number of posts to fetch. -1 for all posts. minimum is 100
    '''
    subreddit = reddit.subreddit(subreddit_name)
    posts = []

    if fetch_hot:
        hot_posts = subreddit.hot(limit=maximum_post)
        posts.extend(hot_posts)
    if fetch_new:
        new_posts = subreddit.new(limit=maximum_post)
        posts.extend(new_posts)
    if fetch_top:
        top_posts = subreddit.top(limit=maximum_post)
        posts.extend(top_posts)
    if fetch_rising:
        rising_posts = subreddit.rising(limit=maximum_post)
        posts.extend(rising_posts)
    if fetch_controversial:
        controversial_posts = subreddit.controversial(limit=maximum_post)
        posts.extend(controversial_posts)

    posts_url = []
    for post in posts:
        posts_url.append(post.url)
    return posts_url

In [33]:

def fetch_post(url: str = "https://www.reddit.com/r/learnpython/comments/1234567/how_to_learn_python/") -> dict:
    '''
    url: url of the reddit post. For example: https://www.reddit.com/r/learnpython/comments/1234567/how_to_learn_python/
    '''
    if not url.startswith("https://www.reddit.com"):
        return {
            'title': None,
            'url': None,
            'flair': None,
            'author': None,
            'selftext': None,
        }
        
    submission = reddit.submission(url=url)
    post_data = {
        'title': submission.title,
        'url': submission.url,
        "flair": submission.link_flair_text,
        'author': str(submission.author),
        # 'created_utc': submission.created_utc,
        # 'num_comments': submission.num_comments,
        # 'score': submission.score,
        'selftext': submission.selftext,
    }
    return post_data



In [34]:
from datetime import date
date = str(date.today()).replace('-', '_')

for subreddit in subreddits:
    print(f"Fetching posts from subreddit: {subreddit}")
    urls = get_url(subreddit, fetch_hot=True, fetch_new=True, maximum_post=100)

    post_dicts = []
    for url in urls:
        post_data = fetch_post(url)
        print(f"Fetched post: {post_data['title']}")
        post_dicts.append(post_data)

    posts_df = pd.DataFrame(post_dicts)
    posts_df.drop_duplicates(subset=['title'], inplace=True)  # Drop duplicates
    posts_df.to_json(f'./{subreddit}/{subreddit}_{date}.json', orient='records', lines=True)
    
    print()

Fetching posts from subreddit: creepypasta
Fetched post: None
Fetched post: Post Creepy Images on r/EyeScream - Our New Subreddit!
Fetched post: They voted for me to die
Fetched post: Does anyone know what creepypasta this image comes from?
Fetched post: Why is the category of Lost Episodes so hated?
Fetched post: ive been looking for this story for.. four years now :/
Fetched post: Trying to find a creepypasta.
Fetched post: Any psychological horror creepypasta recommendations?
Fetched post: My Hero Academia: A Burned Dream
Fetched post: Made this after not sleeping well and reading bad pastas. (Very short)
Fetched post: Help Me Find A Certain Story Please
Fetched post: The Mulch Man
Fetched post: Elios the death kid (...a creepypasta I made..I don't know if it's good)
Fetched post: The Rules for Surviving Our Harbor
Fetched post: I can hear what my reflection is thinking!
Fetched post: Time Won't Let You Cheat It
Fetched post: I Share the Gila Valley with a Kaiju 3
Fetched post: The 